## Interfaces, Protocols, and ABCs

Implementing a full protocol may require several methods, but often it is OK to implement only part of it. Consider the Vowels

In [1]:
class Vowels:
   def __getitem__(self, i):
       return 'AEIOU'[i]

In [2]:
v = Vowels()

In [3]:
v[0], v[-1]

('A', 'U')

In [4]:
for c in v:
    print(c)

A
E
I
O
U


In [5]:
'E' in v, 'Z' in v

(True, False)

Implementing `__getitem__` is enough to allow retrieving items by index, and also to support iteration and the `in` operator.
Eventhough, we don't implement the `__contains__` method. Python is smart enough to know that by implementing `__getitem__`, an object could also inherit from the `__contains__` interface too.

Vowels class does not inherit from `abc.Sequence` and it only implements `__getitem__`.
There is no `__iter__` method, yet Vowels instances are iterable because—as a fallback—if Python finds a `__getitem__` method, it tries to iterate over the object by calling that method with integer indexes starting with 0. Because Python is smart enough to iterate over Vowels instances, it can also make the in operator work even when the `__contains__` method is missing: it does a sequential scan to check if an item is present.

In summary, given the importance of sequence-like data structures, Python manages to make iteration and the in operator work by invoking `__getitem__` when `__iter__` and `__contains__` are unavailable.

### Monkey-Patching

Monkey patching is dynamically changing a module, class, or function at runtime, to add features or fix bugs. For example, the `gevent` networking library monkey patches parts of Python’s standard library to allow lightweight concurrency without threads or `async/await`.

In [7]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

The FrenchDeck class is missing an essential feature: it cannot be shuffled because it doesn't support the `__setitem__` interface. The problem is that shuffle operates in place, by swapping items inside the collection, and FrenchDeck only implements the immutable sequence protocol. Mutable sequences must also provide a `__setitem__` method.

In [8]:
from random import shuffle

deck = FrenchDeck()
shuffle(deck) # This will throw an Error

TypeError: 'FrenchDeck' object does not support item assignment

Because Python is dynamic, we can fix this at runtime, even at the interactive console.

In [12]:
from random import shuffle

deck = FrenchDeck()
deck._cards[:8]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades'),
 Card(rank='5', suit='spades'),
 Card(rank='6', suit='spades'),
 Card(rank='7', suit='spades'),
 Card(rank='8', suit='spades'),
 Card(rank='9', suit='spades')]

In [13]:
def set_card(deck, position, card):
    deck._cards[position] = card

FrenchDeck.__setitem__ = set_card # Monkey-patching objects

shuffle(deck)
deck[:8]

[Card(rank='4', suit='diamonds'),
 Card(rank='4', suit='hearts'),
 Card(rank='8', suit='clubs'),
 Card(rank='5', suit='diamonds'),
 Card(rank='6', suit='spades'),
 Card(rank='3', suit='clubs'),
 Card(rank='K', suit='clubs'),
 Card(rank='4', suit='spades')]

The trick is that `set_card` knows that the deck object has an attribute named `_cards`, and `_cards` must be a mutable sequence. The set_card function is then attached to the FrenchDeck class as the `__setitem__` special method.

## Defensive Programming and “Fail Fast”

Many bugs cannot be caught except at runtime—even in mainstream statically typed languages. In a dynamically typed language, “fail fast” is excellent advice for safer and easier-to-maintain programs. Failing fast means raising runtime errors as soon as possible, for example, rejecting invalid arguments right at the beginning of a function body.

Here is one example: when you write code that accepts a sequence of items to process internally as a list, don’t enforce a list argument by type checking. Instead, take the argument and immediately build a list from it

In [15]:
def __init__(self, iterable):
    self._balls = list(iterable)

That way you make your code more flexible, because the `list()` constructor handles any iterable that fits in memory. If the argument is not iterable, the call will fail fast with a very clear `TypeError` exception, right when the object is initialized. If you want to be more explict, you can wrap the `list()` call with `try/except` to customize the error message.

If you don’t catch the invalid argument in the class constructor, the program will blow up later, when some other method of the class needs to operate on `self._balls` and it is not a `list`. Then the root cause will be harder to find.

Of course, calling `list()` on the argument would be bad if the data shouldn’t be copied, either because it’s too large or because the function, by design, needs to change it in place for the benefit of the caller, like `random.shuffle` does. In that case, a runtime check like `isinstance(x, abc.MutableSequence)` would be the way to go.

If you are afraid to get an infinite generator—not a common issue—you can begin by calling `len()` on the argument. This would reject iterators, while safely dealing with tuples, arrays, and other existing or future classes that fully implement the `Sequence` interface. Calling `len()` is usually very cheap, and an invalid argument will raise an error immediately.

On the other hand, if any iterable is acceptable, then call `iter(x)` as soon as possible to obtain an iterator.

In this case, a type hint could catch some problems earlier, but not all problems. Recall that the type `Any` is consistent-with every other type. Type inference may cause a variable to be tagged with the `Any` type. When that happens, the type checker is in the dark. In addition, type hints are not enforced at runtime. Fail fast is the last line of defense.

Defensive code leveraging duck types can also include logic to handle different types without using `isinstance()` or `hasattr()` tests.

```
try:  1
    field_names = field_names.replace(',', ' ').split()  2
except AttributeError:  3
    pass  4
field_names = tuple(field_names)  5
if not all(s.isidentifier() for s in field_names):  6
    raise ValueError('field_names must all be valid identifiers')
```

1. Assume it’s a string (EAFP = it’s easier to ask forgiveness than permission).

2. Convert commas to spaces and split the result into a list of names.

3. Sorry, field_names doesn’t quack like a str: it has no .replace, or it returns something we can’t .split.

4. If AttributeError was raised, then field_names is not a str and we assume it was already an iterable of names.

5. To make sure it’s an iterable and to keep our own copy, create a tuple out of what we have. A tuple is more compact than list, and it also prevents my code from changing the names by mistake.

6. Use str.isidentifier to ensure every name is valid.

# Goose Typing

Python doesn’t have an `interface` keyword. We use abstract base classes (ABCs) to define interfaces for explicit type checking at runtime—also supported by static type checkers.

What goose typing means is: `isinstance(obj, cls)` is now just fine ... as long as `cls` is an abstract base class—in other words, cls’s metaclass is `abc.ABCMeta`.

In [16]:
class Struggle:
    def __len__(self):
        return 23

from collections import abc
isinstance(Struggle(), abc.Sized)

True

`abc.Sized` recognizes Struggle as “a subclass,” with no need for registration, as implementing the special method named `__len__` is all it takes

To summarize, goose typing entails:

- Subclassing from ABCs to make it explict that you are implementing a previously defined interface.

- Runtime type checking using ABCs instead of concrete classes as the second argument for isinstance and issubclass.

## Subclassing an ABC

In [17]:
from collections import namedtuple, abc

Card = namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):
        self._cards[position] = value

    def __delitem__(self, position):
        del self._cards[position]

    def insert(self, position, value):
        self._cards.insert(position, value)

- `__setitem__` is all we need to enable shuffling…

- but subclassing MutableSequence forces us to implement `__delitem__`, an abstract method of that ABC.

- We are also required to implement `insert`, the third abstract method of `MutableSequence`.

Python does not check for the implementation of the abstract methods at import time (when the frenchdeck2.py module is loaded and compiled), but only at runtime when we actually try to instantiate FrenchDeck2.

To write FrenchDeck2 as a subclass of `MutableSequence`, I had to pay the price of implementing `__delitem__` and insert, which my examples did not require. In return, `FrenchDeck2` inherits five concrete methods from `Sequence`: `__contains__`, `__iter__`, `__reversed__`, index, and count (Since `MutableSequence` is a subclass of `Sequence`). From `MutableSequence`, it gets another six methods: `append`, `reverse`, `extend`, `pop`, `remove`, and `__iadd__` — which supports the += operator for in place concatenation.

### Example

In [18]:
import abc

class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it.

        This method should raise `LookupError` when the instance is empty.
        """

    def loaded(self):
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())

    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(items)

In [19]:
# from tombola import Tombola

class Fake(Tombola):
    def pick(self):
        return 13

In [20]:
Fake

__main__.Fake

In [21]:
Fake()

TypeError: Can't instantiate abstract class Fake with abstract method load

`TypeError` is raised when we try to instantiate `Fake`. The message is very clear: `Fake` is considered abstract because it failed to implement `load`, one of the abstract methods declared in the `Tombola` ABC.

## ABC Syntax Details

In [25]:
class MyABC(abc.ABC):
    @classmethod
    @abc.abstractmethod
    def an_abstract_classmethod(cls):
        pass

# No other decorator may appear between @abstractmethod and the def statement.

## A Virtual Subclass of an ABC

An essential characteristic of goose typing—and one reason why it deserves a waterfowl name is the ability to register a class as a virtual subclass of an ABC, even if it does not inherit from it. When doing so, we promise that the class faithfully implements the interface defined in the ABC—and Python will believe us without checking. If we lie, we’ll be caught by the usual runtime exceptions.

This is done by calling a `register` class method on the ABC. The registered class then becomes a virtual subclass of the ABC, and will be recognized as such by `issubclass`, but it does not inherit any methods or attributes from the ABC.

In [26]:
from random import randrange

# from tombola import Tombola

@Tombola.register
class TomboList(list):

    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend

    def loaded(self):
        return bool(self)

    def inspect(self):
        return tuple(self)

# Tombola.register(TomboList)

`Tombolist` is registered as a virtual subclass of `Tombola` and it extends a `list`. When calling `issubclass(TomboList, Tombola)`, the call will return True due to Vitual Subclassing even the `Tombolist` doesn't fully implement the `load` abstract method of the `Tombola`.

In [27]:
issubclass(TomboList, Tombola)

True

In [28]:
t = TomboList(range(100))

In [29]:
isinstance(t, Tombola)

True

Subclassing an ABC or registering with an ABC are both explicit ways of making our classes pass `issubclass` checks — as well as `isinstance` checks, which also rely on `issubclass`. But some ABCs support structural typing as well.

## Structural Typing with ABCs

ABCs are mostly used with nominal typing. When a class Sub explicitly inherits from `AnABC`, or is registered with `AnABC`, the name of `AnABC` is linked to the Sub class—and that’s how at runtime, `issubclass(AnABC, Sub)` returns True.

In contrast, structural typing is about looking at the structure of an object’s public interface to determine its type: an object is consistent-with a type if it implements the methods defined in the type. Dynamic and static duck typing are two approaches to structural typing.

It turns out that some ABCs also support structural typing

In [30]:
class Struggle:
    def __len__(self): return 23

from collections import abc

isinstance(Struggle(), abc.Sized), issubclass(Struggle, abc.Sized)

(True, True)

Class `Struggle` is considered a subclass of `abc.Sized` by the `issubclass` function (and, consequently, by isinstance as well) because `abc.Sized` implements a special class method named `__subclasshook__`.

The `__subclasshook__` for `Sized` checks whether the class argument has an attribute named _`_len__`. If it does, then it is considered a virtual subclass of `Sized`.

In [31]:
# Sized's Source code
from abc import ABCMeta, abstractmethod


class Sized(metaclass=ABCMeta):

    __slots__ = ()

    @abstractmethod
    def __len__(self):
        return 0

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Sized:
            if any("__len__" in B.__dict__ for B in C.__mro__): # 1
                return True  # 2
        return NotImplemented # 3

1. If there is an attribute named `__len__` in the `__dict__` of any class listed in `C.__mro__` (i.e., `C` and its superclasses) ...
2. ... return True, signaling that `C` is a virtual subclass of `Sized`.
3. Otherwise, return `NotImplemented` to let the subclass check proceed.

That’s how `__subclasshook__` allows ABCs to support structural typing. You can formalize an interface with an ABC, you can make `isinstance` checks against that ABC, and still have a completely unrelated class pass an `issubclass` check because it implements a certain method (or because it does whatever it takes to convince a `__subclasshook__` to vouch for it).